# Gesture Analysis

In [3]:
# %pip install opencv-python mediapipe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import cv2
import mediapipe as mp

def analyze_gestures(video_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    valid_frame_count = 0
    gesture_movement = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)

        if result.pose_landmarks:
            lm = result.pose_landmarks.landmark
            required = [
                mp_pose.PoseLandmark.LEFT_SHOULDER,
                mp_pose.PoseLandmark.LEFT_WRIST,
                mp_pose.PoseLandmark.RIGHT_SHOULDER,
                mp_pose.PoseLandmark.RIGHT_WRIST
            ]

            if all(lm[i].visibility > 0.5 for i in required):
                def dist(a, b):
                    return np.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)

                l_dist = dist(lm[mp_pose.PoseLandmark.LEFT_SHOULDER], lm[mp_pose.PoseLandmark.LEFT_WRIST])
                r_dist = dist(lm[mp_pose.PoseLandmark.RIGHT_SHOULDER], lm[mp_pose.PoseLandmark.RIGHT_WRIST])
                gesture_movement.append((frame_count, l_dist, r_dist))
                valid_frame_count += 1

    cap.release()
    pose.close()

    if not gesture_movement:
        return {
            "Error": "Недостаточно кадров с хорошо видимыми жестами"
            
        }

    gesture_movement = np.array(gesture_movement)
    left_var = np.var(gesture_movement[:, 1])
    right_var = np.var(gesture_movement[:, 2])
    avg_gesture_amp = np.mean((gesture_movement[:, 1] + gesture_movement[:, 2]) / 2)

    return {
        "left_hand_variability": round(left_var, 5),
        "right_hand_variability": round(right_var, 5),
        "avg_hand_movement_amplitude": round(avg_gesture_amp, 5),
        "valid_frames_analyzed": valid_frame_count,
        "total_frames": frame_count,
        "coverage_%": round(100 * valid_frame_count / frame_count, 1)
    }


In [5]:
video_path = "data\\fragments\\videos\\8Dv2Hdf5TRg\\8Dv2Hdf5TRg_seg000.mp4"
result = analyze_gestures(video_path)

In [6]:
result

{'left_hand_variability': 0.00914,
 'right_hand_variability': 0.01406,
 'avg_hand_movement_amplitude': 0.37095,
 'valid_frames_analyzed': 366,
 'total_frames': 733,
 'coverage_%': 49.9}

In [11]:
data = pd.read_excel("data/fragments/dataset.xlsx")

features = ['left_hand_variability', 'right_hand_variability', 'avg_hand_movement_amplitude', 'valid_frames_analyzed', 'total_frames', 'coverage_%']
data[features] = np.float64(0)

In [ ]:
# import os
# 
# videos = "data\\fragments\\videos"
# for video in os.listdir(videos):
#     video_path = os.path.join(videos, video)
#     for segment in os.listdir(video_path):
#         segment_path = os.path.join(video_path, segment)
#         result = analyze_gestures(segment_path)
#         
#         if "Error" in result:
#             print(f"Error in {segment_path}: {result['Error']}")
#             continue
#         
#         data.loc[data['segment_name'] == segment[:-4], features] = list(result.values())
# 
# data.to_csv("dataset_gesture.csv", index=False)

In [30]:
# with open("C:\\Users\\User\\OneDrive - НИУ Высшая школа экономики\\Рабочий стол\\output.txt", 'r') as f:
#     lines = f.readlines()
# 
# video_with_errors = [line.split()[2][:-1].split('\\')[-1][:-4] for line in lines]
# 
# data = data[~data['segment_name'].isin(video_with_errors)]

In [4]:
data = pd.read_csv("D:\\course_project\\data\\datasets\\dataset_gesture_cleaned.csv")
data.columns

Index(['segment_name', 'confidence_assessment',
       'mediapipe_left_hand_variability', 'mediapipe_right_hand_variability',
       'mediapipe_avg_hand_movement_amplitude',
       'mediapipe_valid_frames_analyzed', 'mediapipe_total_frames',
       'mediapipe_coverage_%'],
      dtype='object')

In [3]:
mediapipe = ['left_hand_variability', 'right_hand_variability', 'avg_hand_movement_amplitude', 'valid_frames_analyzed', 'total_frames', 'coverage_%']
mediapipe = ['mediapipe_' + i for i in mediapipe]

new_columns = ['segment_name', 'confidence_assessment'] + mediapipe
data.columns = new_columns

data.to_csv("D:\\course_project\\data\\datasets\\dataset_gesture_cleaned.csv", index=False)

---

In [13]:
import cv2
import mediapipe as mp

# Initialize MediaPipe pose and drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Load your video file
video_path = 'data/fragments/videos/Ks-_Mh1QhMc/Ks-_Mh1QhMc_seg035.mp4'  # <- Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Optional: get video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('pose_output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Get pose landmarks
        results = pose.process(image)

        # Convert back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks and connections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
            )

        # Write the frame to output file
        out.write(image)

        # Optionally show the frame
        cv2.imshow('MediaPipe Pose on Video', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
out.release()
cv2.destroyAllWindows()
